<a href="https://colab.research.google.com/github/sunyeul/ToyProjectLab/blob/feature%2Fjapanese-alpaca-fine-tuning/Japanese_Alpaca_Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/assets/logo.png)

## Alpacaの概要

- Alpacaは、MetaのLLaMA 7Bモデルから微調整され、OpenAIのtext-davinci-003に似た性能を示す

- Alpacaは小型で再現が容易かつ安価

- Alpacaは学術研究専用で、商用利用は禁止されている

    - 商用利用禁止の理由は、LLaMAの非営利ライセンス継承、OpenAIの利用規約による競合禁止、十分な安全対策がないため。

![トレーニングレシピ](https://crfm.stanford.edu/static/img/posts/2023-03-13-alpaca/alpaca_main.jpg)

In [ ]:
!nvidia-smi

Mon May  8 07:08:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## ダウンロードソースコード


- https://github.com/tloen/alpaca-lora.git
- peftバグ：https://github.com/tloen/alpaca-lora/issues/293

In [ ]:
!git clone -qq https://github.com/tloen/alpaca-lora.git
%cd alpaca-lora
!pip install -r requirements.txt -qq
!pip uninstall peft -y -qq
!pip install -qq git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08

fatal: destination path 'alpaca-lora' already exists and is not an empty directory.
/content/alpaca-lora
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### 原本のデータセット

- Fine-tuning用に使われた命令追従型の学習データ
    - https://github.com/tatsu-lab/stanford_alpaca/blob/main/alpaca_data.json
    - https://huggingface.co/datasets/tatsu-lab/alpaca/viewer/tatsu-lab--alpaca

    - instruction: str - モデルが実行すべきタスクを記述。52Kの指示はユニーク

    - input: str - タスクのオプションのコンテキストまたは入力。たとえば、指示が「次の記事を要約」の場合、入力は記事。例の約40%には入力がある

    - output: str - 「text-davinci-003」によって生成された指示に対する回答

```JSON
[
    ...,
    {
        "instruction": "Give three tips for staying healthy.",
        "input": "",
        "output": "1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule."
    },
    {
        "instruction": "Explain why the following fraction is equivalent to 1/4",
        "input": "4/16",
        "output": "The fraction 4/16 is equivalent to 1/4 because both numerators and denominators are divisible by 4. Dividing both the top and bottom numbers by 4 yields the fraction 1/4."
    },
    ...
]
```

### 和訳データセット

- ChatGPT翻訳： https://raw.githubusercontent.com/masa3141/japanese-alpaca-lora/main/data/japanese_alpaca_data.json

```JSON
[
    ...,
    {
        "instruction": "「健康を維持するための3つのヒントを教えてください。」",
        "input": "",
        "output": "「バランスの良い食事を摂り、果物や野菜をたっぷりと含めるようにしてください。\n2. 定期的に運動して、体を活発で強く保ちましょう。\n3. 十分な睡眠をとり、一定の睡眠スケジュールを維持してください。」"
    },
    {
        "instruction": "「以下の分数が1/4と等しい理由を説明してください。」",
        "input": "'4/16'は日付を表しています。",
        "output": "「4/16の分数は、両方の分子と分母が4で割り切れるため、1/4と等しいです。上部と下部の数値を両方とも4で割ると、1/4の分数が得られます。」"
    },
    ...
]
```

- https://raw.githubusercontent.com/shi3z/alpaca_ja/main/alpaca_data_ja.json


```JSON
[
    ...,
    {
        "instruction":  "健康維持のための3つのコツを教えてください。\n",
        "input":"\n",
        "output":"1、バランスのとれた食事を摂り、野菜や果物を十分に摂ること。2、定期的に運動をして体の活力を保つこと。3、睡眠時間を十分にとり、規則正しい睡眠をとること。\n"
    },
    {
        "instruction":  "次の分数が1/4に相当する理由を説明せよ。\n",
        "input":"4/16個\n",
        "output":"分数4/16は1/4に相当します。これは、分子と分母の両方が4で割り切れるためです。上の数値と下の数値の両方を4で割ると、分数1/4が得られます。\n"
    },
    ...
]
```

In [ ]:
!wget -q https://raw.githubusercontent.com/masa3141/japanese-alpaca-lora/main/data/japanese_alpaca_data.json

import json

with open('japanese_alpaca_data.json', 'r', encoding='UTF-8') as f:
    dataset = json.load(f)

dataset[:5]

[{'instruction': '「健康を維持するための3つのヒントを教えてください。」',
  'input': '',
  'output': '「バランスの良い食事を摂り、果物や野菜をたっぷりと含めるようにしてください。\n2. 定期的に運動して、体を活発で強く保ちましょう。\n3. 十分な睡眠をとり、一定の睡眠スケジュールを維持してください。」'},
 {'instruction': '「三原色とは何ですか？」', 'input': '', 'output': '「三原色は赤、青、黄色です。」'},
 {'instruction': '「原子の構造を説明してください。」',
  'input': '',
  'output': '「原子は、陽子と中性子を含む核と、核の周りを軌道を描いて移動する電子から構成されています。陽子と中性子は正の電荷を持ち、電子は負の電荷を持っており、全体としては中性の原子になります。各粒子の数が原子番号と原子の種類を決定します。」'},
 {'instruction': '「どうすれば大気汚染を減らせますか？」',
  'input': '',
  'output': '「再生可能エネルギー源への移行、公共交通機関の利用促進、化石燃料の燃焼禁止、産業源からの排出削減政策の実施、車両排出基準の実施など、大気汚染を減らすための方法はいくつかあります。さらに、個人は車の利用を減らしたり、木材の燃焼を避けたり、省エネ家電に切り替えたりすることで、大気汚染を減らすことができます。」'},
 {'instruction': '「あなたが困難な決断をしなければならなかった時について説明してください。」',
  'input': '',
  'output': '私は建設会社でプロジェクトマネージャーとして働いていた時、困難な決断をしなければならない状況に陥りました。私が担当していたプロジェクトは、クライアントの期待に応えるためにある特定の日までに完成しなければならなかったのですが、予期せぬ遅延が発生し、期限に間に合わなくなってしまいました。そこで、私は困難な決断を下すことになりました。私は期限を延長することを決めましたが、チームのリソースをさらに使い果たし、予算を増やす必要がありました。リスクのある決断でしたが、プロジェクトを期限通りに完了し、

## カスタムプロンプト

- 原本のプロンプト

```JSON
{
    "description": "Template used by Alpaca-LoRA.",
    "prompt_input": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
    "prompt_no_input": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n",
    "response_split": "### Response:"    
}
```

- 和駅のカスタムプロンプト

```JSON
{
    "description": "Alpaca-LoRAが使用するテンプレート。",
    "prompt_input": "以下に、タスクを説明する指示と、さらなる文脈を提供する入力がペアになっています。要求に適切に対応する回答を記述してください。\n\n### 指示:\n{instruction}\n\n### 入力:\n{input}\n\n### 回答:\n",
    "prompt_no_input": "以下に、タスクを説明する指示があります。要求に適切に対応する回答を記述してください。\n\n### 指示:\n{instruction}\n\n### 回答:\n",
    "response_split": "### 回答:"
}
```

In [ ]:
import json

prompt_template = {
    "description": "Alpaca-LoRA カスタムテンプレート",
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n"
        "以下に、タスクを説明する指示と、さらなる文脈を提供する入力がペアになっています。\n"
        "Write a response that appropriately completes the request.\n要求に適切に対応する回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 入力:\n{input}\n\n### 回答:\n"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n"
        "以下に、タスクを説明する指示があります。\n\n"
        "Write a response that appropriately completes the request.\n要求に適切に対応する回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 回答:\n"
    ),
    "response_split": "### 回答:",
}

with open('templates/custom.json', 'w', encoding='utf-8') as f:
    json.dump(prompt_template, f, ensure_ascii=False)

## Fine-Tuning

- https://github.com/tloen/alpaca-lora/blob/0e1a5d52a460d14aea2325e43c302972badb9cdd/finetune.py#L28
- https://huggingface.co/blog/peft

- 基本設定の時：
    - 3h/1epoch in NVIDIA A100
    - 12h/1epoch in NVIDIA T4

- 注意事項
    - 5epoch以上が精度が出やすい
    - 学習率はデフォルトで3e-4. データセットによって要調整

In [ ]:
!python finetune.py \
    --base_model 'decapoda-research/llama-7b-hf' \
    --data_path 'japanese_alpaca_data.json' \
    --output_dir './output' \
    --num_epochs 1 \
    --learning_rate 5e-4 \
    --val_set_size 2000 \
    --batch_size 512 \
    --micro_batch_size 16 \
    --prompt_template_name 'custom'

In [ ]:
# ckptをローカルに保存したい場合
save_local: bool = False

if save_local:
    from google.colab import drive
    drive.mount('/content/drive')
    !mkdir /content/drive/MyDrive/LLaMa-Alpaca-LoRA
    !cp -a output /content/drive/MyDrive/LLaMa-Alpaca-LoRA

## 文章の生成

- Gradioデモ：https://jalpaca.infertron.com/

In [ ]:
import torch
from peft import PeftModel
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")

model = LlamaForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(
    model,
    "kunishou/Japanese-Alpaca-LoRA-7b-v0",
    torch_dtype=torch.float16,
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:
PROMPT = """
以下に、タスクを説明する指示と、さらなる文脈を提供する入力がペアになっています。
### 指示:
アルパカについて教えてください.
### 回答:
"""

inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
)
input_ids = inputs["input_ids"].cuda()

generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=4.8,
)

print("Generating...")
generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=256,
)

for s in generation_output.sequences:
    print(tokenizer.decode(s))

Generating...
 ⁇  
以下に、タスクを説明する指示と、さらなる文脈を提供する入力がペアになっています。
### 指示:
アルパカについて教えてください.
### 回答:
Alpacas are native to South America, and they're known for their soft wool that can be spun into yarn or knitted with needles! They also have a very distinctive face shape - the ears point upward like an elf ear while being rounded on top (like Mickey Mouse). Alongside sheep & goats in farms around Peru/Bolivia where alpaaca herds live today; these animals were once used as pack mules by Incan people who lived there long ago before Europeans arrived...
